<a href="https://colab.research.google.com/github/DanteAgarwal/how-uber-work/blob/main/major_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# creating dataset from kaggle 

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"akshatishere","key":"e36cfa35f4c26b50aa580b2d3b35f5cf"}'}

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d plameneduardo/sarscov2-ctscan-dataset

 99% 228M/230M [00:06<00:00, 47.0MB/s]
100% 230M/230M [00:06<00:00, 36.1MB/s]


In [ ]:
!mkdir ctscans 
!unzip sarscov2-ctscan-dataset.zip -d ctscans

Archive:  sarscov2-ctscan-dataset.zip
  inflating: ctscans/COVID/Covid (1).png  
  inflating: ctscans/COVID/Covid (10).png  
  inflating: ctscans/COVID/Covid (100).png  
  inflating: ctscans/COVID/Covid (1000).png  
  inflating: ctscans/COVID/Covid (1001).png  
  inflating: ctscans/COVID/Covid (1002).png  
  inflating: ctscans/COVID/Covid (1003).png  
  inflating: ctscans/COVID/Covid (1004).png  
  inflating: ctscans/COVID/Covid (1005).png  
  inflating: ctscans/COVID/Covid (1006).png  
  inflating: ctscans/COVID/Covid (1007).png  
  inflating: ctscans/COVID/Covid (1008).png  
  inflating: ctscans/COVID/Covid (1009).png  
  inflating: ctscans/COVID/Covid (101).png  
  inflating: ctscans/COVID/Covid (1010).png  
  inflating: ctscans/COVID/Covid (1011).png  
  inflating: ctscans/COVID/Covid (1012).png  
  inflating: ctscans/COVID/Covid (1013).png  
  inflating: ctscans/COVID/Covid (1014).png  
  inflating: ctscans/COVID/Covid (1015).png  
  inflating: ctscans/COVID/Covid (1016).png  
  i

# importing important modeles of python


In [ ]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt


import sys
import random
import warnings
warnings.filterwarnings('ignore')

import cv2
import os
from PIL import Image
import PIL
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

from scipy.signal import convolve2d
from skimage.segmentation import slic,mark_boundaries
from pylab import rcParams


## preprocessing of dataset images 
### 1. clahe Algo (for contrast) .
### 2. cropping of image .
### 3. main driver code .
### (also we maked full data set in 200,200 resloution and grayscale .

In [ ]:
def clahe_enhancer(img,demo=False):
    img = np.uint8(img*255)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    clahe_img = clahe.apply(img)
    if demo:
        img_flattened = img.flatten()
        clahe_img_flattened = clahe_img.flatten()
        fig = plt.figure()
        rcParams['figure.figsize'] = 10,10

        plt.subplot(2, 2, 1)
        plt.imshow(img, cmap='bone')
        plt.title("Original CT-Scan")

        plt.subplot(2, 2, 2)
        plt.hist(img_flattened)
        plt.title("Histogram of Original CT-Scan")

        plt.subplot(2, 2, 3)
        plt.imshow(clahe_img, cmap='bone')
        plt.title("CLAHE Enhanced CT-Scan")

        plt.subplot(2, 2, 4)
        plt.hist(clahe_img_flattened)
        plt.title("Histogram of CLAHE Enhanced CT-Scan")

    return clahe_img

In [ ]:
import imutils
def cropper1(img):
   
  grassuian=cv2.GaussianBlur(img,(5,5),0)
  threshold_img=cv2.threshold(grassuian,45,255,cv2.THRESH_BINARY)[1]
  threshold_img = cv2.erode(threshold_img,None,iterations=2)
  threshold_img = cv2.dilate(threshold_img,None,iterations=2)

  contour = cv2.findContours(threshold_img.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
  contour = imutils.grab_contours(contour)
  c=max(contour, key=cv2.contourArea)

  extreme_pnts_left=tuple(c[c[:,:,0].argmin()][0])
  extreme_pnts_right=tuple(c[c[:,:,0].argmax()][0])
  extreme_pnts_top=tuple(c[c[:,:,1].argmin()][0])
  extreme_pnts_bottom=tuple(c[c[:,:,1].argmax()][0])

  new_img =img[extreme_pnts_top[1]:extreme_pnts_bottom[1],extreme_pnts_left[0]:extreme_pnts_right[0]]
  return new_img

In [ ]:
from google.colab.patches import cv2_imshow
import glob

imdir = '/content/ctscans/non-COVID/'
ext = ['png']
img_list = []
files = []
cout=0
[files.extend(glob.glob(imdir + '*.' + e)) for e in ext]
for file in files:
  img = cv2.imread(file)
  img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)                         
  img1 = cv2.resize(img,(200,200),interpolation=cv2.INTER_AREA)
  Clahe_img1 = clahe_enhancer(img1)
  img1=cropper1(Clahe_img1)
  cout +=1
  cv2.imwrite('/content/ctscans-preprocessed/Non_covid_preprocessed/Non_covid_preposed_{0}.png'.format(cout), img1)
  
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
from google.colab.patches import cv2_imshow
import glob
imdir = '/content/ctscans/COVID/'
ext = ['png']
img_list = []
files = []
cout=0
[files.extend(glob.glob(imdir + '*.' + e)) for e in ext]
for file in files:
  img = cv2.imread(file)
  img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)                         
  img1 = cv2.resize(img,(200,200),interpolation=cv2.INTER_AREA)
  Clahe_img1 = clahe_enhancer(img1)
  img1=cropper1(Clahe_img1)
  cout +=1
  cv2.imwrite("/content/ctscans-preprocessed/covid_preprocessed/covid_preposed_{0}.png".format(cout), img1)

cv2.waitKey()
cv2.destroyAllWindows()

# feature extraction 
### 1